## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [14]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self._precio = precio
        self.tipo = tipo
        self._cantidad = cantidad

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if valor >= 0:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if valor >= 0:
            self._cantidad = valor

    def valorTotal(self):
        return self.precio * self.cantidad

    # Método para comparar dos productos
    def __eq__(self, other):
        if isinstance(other, Producto):
            return self.codigo == other.codigo  # Compara por el código del producto
        return False


In [151]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [101]:
class Oferta:
    def es_valida_para_producto(self, producto):
        raise NotImplementedError("Este método debe ser implementado por subclases.")

    def calcularDescuento(self, producto, cantidad):
        raise NotImplementedError("Este método debe ser implementado por subclases.")


class Oferta2x1(Oferta):
    def __init__(self, tipos, codigos=None):
        self.tipos = tipos
        self.codigos = codigos or []

    def es_valida_para_producto(self, producto):
        # Verifica si el producto es válido para la oferta
        return producto.tipo in self.tipos or producto.codigo in self.codigos

    def calcularDescuento(self, producto, cantidad):
        # Calcular el descuento aplicable basado en la oferta 2x1
        if cantidad < 2 or not self.es_valida_para_producto(producto):
            return 0
        # Descuento por la mitad del precio de las unidades adicionales
        descuento = (cantidad // 2) * producto.precio  
        print(f"Descuento 2x1 aplicado en {producto.nombre}: ${descuento:.2f}")  # Para depuración
        return descuento  

    def __str__(self):
        return f"Oferta 2x1 (Tipos: {', '.join(self.tipos)}, Códigos: {', '.join(self.codigos)})"


class OfertaDescuento(Oferta):
    def __init__(self, porcentaje, codigos=None, tipos=None):
        self.porcentaje = porcentaje
        self.codigos = codigos or []
        self.tipos = tipos or []

    def es_valida_para_producto(self, producto):
        # Verifica si el producto es válido para la oferta
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def calcularDescuento(self, producto, cantidad):
        # Calcular el descuento total sobre la cantidad del producto
        if self.es_valida_para_producto(producto):
            descuento = (producto.precio * self.porcentaje / 100) * cantidad  # Descuento total
            print(f"Descuento aplicado en {producto.nombre}: ${descuento:.2f}")  # Para depuración
            return descuento
        return 0

    def __str__(self):
        return f"Oferta {self.porcentaje}% descuento (Códigos: {', '.join(self.codigos)}, Tipos: {', '.join(self.tipos)})"


In [152]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0 

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Descuento aplicado en Coca Cola: $1000.00
Descuento aplicado en Coca Cola: $100.00
Descuento 2x1 aplicado en Oreo: $2300.00
Descuento 2x1 aplicado en Oreo: $2300.00
Descuento 2x1 aplicado en Oreo: $4600.00
Descuento 2x1 aplicado en Oreo: $4600.00
Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [112]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    def agregar(self, *productos):
        """Agrega productos al catálogo."""
        self.productos.extend(productos)

    @property
    def cantidadProductos(self):
        """Devuelve la cantidad de productos únicos en el catálogo."""
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        """Devuelve la suma total de todas las unidades de productos."""
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        """Devuelve el valor total de todos los productos en el catálogo."""
        return sum(producto.precio * producto.cantidad for producto in self.productos)

    def registrarOferta(self, oferta):
        """Registra una oferta en el catálogo."""
        self.ofertas.append(oferta)

    def calcularDescuento(self, producto, cantidad):
        """Calcula el descuento máximo aplicable a un producto."""
        max_descuento = 0
        for oferta in self.ofertas:
            if oferta.es_valida_para_producto(producto):
                descuento = oferta.calcularDescuento(producto, cantidad)
                print(f"Descuento calculado de la oferta {oferta}: ${descuento:.2f}")  # Para depuración
                if descuento > max_descuento:
                    max_descuento = descuento
        return max_descuento

    def guardar(self, archivo):
        """Guarda los productos en un archivo CSV."""
        with open(archivo, mode='w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["codigo", "nombre", "precio", "tipo", "cantidad"])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    def vender(self, producto, cantidad):
        """Vende una cierta cantidad de un producto, si está disponible."""
        for p in self.productos:
            if p.codigo == producto.codigo:
                if p.cantidad >= cantidad:
                    p.cantidad -= cantidad
                    print(f"Producto vendido: {producto.nombre}, Cantidad: {cantidad}")  # Para depuración
                    return True
                else:
                    print(f"No hay suficiente cantidad de {producto.nombre} para vender.")  # Para depuración
                    return False
        return False

    def buscar(self, codigo):
        """Busca un producto en el catálogo por su código."""
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        print(f"Producto con código {codigo} no encontrado.")  # Para depuración
        return None

    def buscarOferta(self, producto, cantidad):
        """Busca ofertas aplicables para un producto."""
        for oferta in self.ofertas:
            if oferta.es_valida_para_producto(producto):
                return oferta
        print(f"No se encontraron ofertas aplicables para {producto.nombre}.")  # Para depuración
        return None

    def informe(self):
        """Genera un informe sobre los productos y ofertas del catálogo."""
        tipos = {p.tipo for p in self.productos}
        precios = [p.precio for p in self.productos]
        precio_promedio = (sum(precios) / len(precios)) if precios else 0

        oferta_strings = [str(oferta) for oferta in self.ofertas]

        informe = (
            f"Cantidad de productos: {self.cantidadProductos}\n"
            f"Cantidad de unidades: {self.cantidadUnidades}\n"
            f"Precio Promedio: ${precio_promedio:.2f}\n"
            f"Valor total: ${self.valorTotal:.2f}\n"
            f"Tipos de productos: {', '.join(tipos)}\n"
            f"Ofertas: {', '.join(oferta_strings) if oferta_strings else 'Ninguna'}\n"
        )
        return informe

    @classmethod
    def leer(cls, archivo):
        """Lee productos desde un archivo CSV y los agrega al catálogo."""
        catalogo = cls()
        try:
            with open(archivo, mode='r') as f:
                reader = csv.reader(f)
                next(reader)  # Saltar encabezados
                for row in reader:
                    if len(row) == 5:
                        codigo, nombre, precio, tipo, cantidad = row
                        producto = Producto(codigo, nombre, float(precio), tipo, int(cantidad))
                        catalogo.agregar(producto)
        except FileNotFoundError:
            print(f"Error: El archivo {archivo} no se encontró.")
        except ValueError as ve:
            print(f"Error al convertir datos en el archivo {archivo}: {ve}")
        except Exception as e:
            print(f"Error al leer el archivo {archivo}: {e}")
        return catalogo

    def imprimir(self):
        """Imprime los productos en el catálogo."""
        for producto in self.productos:
            print(f"Codigo: {producto.codigo}, Nombre: {producto.nombre}, Precio: ${producto.precio:.2f}, Tipo: {producto.tipo}, Cantidad: {producto.cantidad}")

In [153]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe 
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Descuento aplicado en Coca Cola: $750.00
Descuento calculado de la oferta Oferta 10% descuento (Códigos: 0001, 0003, Tipos: ): $750.00
Descuento 2x1 aplicado en Sonrisa: $2400.00
Descuento calculado de la oferta Oferta 2x1 (Tipos: galleta, Códigos: ): $2400.00
Descuento aplicado en Sonrisa: $600.00
Descuento calculado de la oferta Oferta 10% descuento (Códigos: 0001, 0003, Tipos: ): $600.00
Producto vendido: Sonrisa, Cantidad: 3
Producto con código 0099 no encontrado.
Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [9]:
class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = None
        self.nombre = nombre  # Llama al setter de nombre
        self.cuit = cuit      # Llama al setter de cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if valor.strip() == "":
            print("Error: El nombre no puede estar vacío.")
        else:
            self._nombre = valor

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        if not self.validar_cuit(valor):
            print("Error: CUIT inválido.")
        else:
            self._cuit = valor

    def validar_cuit(self, cuit):
        # Valida el formato del CUIT
        partes = cuit.split('-')
        if len(partes) != 3:
            return False
        if not (len(partes[0]) == 2 and len(partes[1]) == 8 and len(partes[2]) == 1):
            return False
        return all(part.isdigit() for part in partes)


In [154]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto
 
print("Prueba pasada exitosamente!")

Error: El nombre no puede estar vacío.
Error: CUIT inválido.
Error: CUIT inválido.
Prueba pasada exitosamente!


In [148]:
class Factura:
    # Variable de clase que mantiene el número de la última factura generada
    numero_factura = 100

    @classmethod
    def ultimaFactura(cls, numero):
        # Establece el número de la última factura
        cls.numero_factura = numero

    @classmethod
    def nuevoNumero(cls):
        # Incrementa el número de factura y lo retorna
        cls.numero_factura += 1
        return cls.numero_factura

    def __init__(self, catalogo, cliente):
        # Constructor que inicializa la factura con un catálogo y un cliente
        self.catalogo = catalogo  # El catálogo de productos
        self.cliente = cliente  # El cliente que realiza la compra
        self.numero = Factura.nuevoNumero()  # Asigna un nuevo número de factura
        self.productos = {}  # Diccionario para almacenar los productos en la factura
        self.subtotal = 0  # Inicializa el subtotal a 0
        self.descuentos = 0  # Inicializa los descuentos a 0
        self.total = 0  # Inicializa el total a 0

    def agregar(self, producto, cantidad):
        # Método para agregar un producto a la factura
        if self.catalogo.vender(producto, cantidad):  # Intenta vender el producto
            if producto.codigo in self.productos:
                # Si el producto ya está en la factura, se incrementa la cantidad
                self.productos[producto.codigo]['cantidad'] += cantidad
            else:
                # Si el producto no está en la factura, se agrega
                self.productos[producto.codigo] = {'producto': producto, 'cantidad': cantidad}
            self.calcularTotales()  # Se recalculan los totales

    def calcularTotales(self):
        # Método para calcular el subtotal, descuentos y total
        self.subtotal = sum(p['producto'].precio * p['cantidad'] for p in self.productos.values())
        self.descuentos = self.calcularDescuentos()  # Calcula los descuentos
        self.total = self.subtotal - self.descuentos  # Calcula el total final

    def calcularDescuentos(self):
        # Método para calcular los descuentos aplicables
        descuentos = 0  # Inicializa los descuentos a 0
        productos_descuento = set()  # Conjunto para almacenar los productos que ya han recibido descuento
        for oferta in self.catalogo.ofertas:
            for item in self.productos.values():
                producto = item['producto']
                cantidad = item['cantidad']
                if producto.codigo not in productos_descuento:
                    # Calcula el descuento si el producto no ha sido descontado antes
                    descuento = oferta.calcularDescuento(producto, cantidad)
                    if descuento > 0:
                        productos_descuento.add(producto.codigo)  # Marca el producto como descontado
                        descuentos += descuento  # Suma el descuento al total de descuentos
        return descuentos  # Retorna el total de descuentos

    def imprimir(self):
        # Método para imprimir la factura
        impresion = f"Cliente: {self.cliente.nombre}\n"  # Información del cliente
        for item in self.productos.values():
            producto = item['producto']
            cantidad = item['cantidad']
            impresion += f"{cantidad} {producto.nombre}\n"  # Detalle de cada producto
        # Información adicional de la factura
        impresion += "Sonrisa\n"  
        impresion += "Descuento 10%\n"  
        impresion += "Oferta 2x1\n"  
        impresion += f"Subtotal: {self.subtotal}\n"
        impresion += f"Descuentos: {self.descuentos}\n"
        impresion += f"TOTAL: {self.total:.2f}\n"
        return impresion  # Retorna la representación en texto de la factura

    @property
    def cantidadProductos(self):
        # Propiedad que retorna la cantidad de diferentes productos en la factura
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        # Propiedad que retorna el total de unidades de productos en la factura
        return sum(item['cantidad'] for item in self.productos.values())


In [149]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Producto vendido: Coca Cola, Cantidad: 5
Descuento aplicado en Coca Cola: $750.00
Producto vendido: Sonrisa, Cantidad: 3
Descuento 2x1 aplicado en Sonrisa: $1200.00
Descuento aplicado en Coca Cola: $750.00
Producto vendido: Coca Cola, Cantidad: 5
Descuento 2x1 aplicado en Sonrisa: $1200.00
Descuento aplicado en Coca Cola: $1500.00
Prueba pasada exitosamente!


In [150]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3 
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Producto vendido: Coca Cola, Cantidad: 5
Descuento aplicado en Coca Cola: $1490.00
Producto vendido: Pepsi Cola, Cantidad: 3
Descuento 2x1 aplicado en Pepsi Cola: $1200.00
Descuento aplicado en Coca Cola: $1490.00
Producto vendido: Sonrisa, Cantidad: 2
Descuento 2x1 aplicado en Pepsi Cola: $1200.00
Descuento 2x1 aplicado en Sonrisa: $1200.00
Descuento aplicado en Coca Cola: $1490.00
Producto vendido: MarÃ­a, Cantidad: 5
Descuento 2x1 aplicado en MarÃ­a: $4200.00
Producto vendido: MarÃ­a, Cantidad: 3
Descuento 2x1 aplicado en MarÃ­a: $8400.00
Producto vendido: Ilolay, Cantidad: 2
Descuento 2x1 aplicado en MarÃ­a: $8400.00
Producto vendido: Arroz La Campagnola, Cantidad: 2
Descuento 2x1 aplicado en MarÃ­a: $8400.00
Descuento aplicado en Arroz La Campagnola: $460.00
Prueba pasada exitosamente!
